In [1]:
# plotting.py
import h5py
import numpy as np
from synthesizer.conversions import lnu_to_absolute_mag
import pandas as pd
import unyt
from unyt import erg, Hz, s
import cmasher as cmr
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import sys
import glob

sys.path.append("/home/jovyan/camels/proj1/")
from setup_params import get_photometry, get_luminosity_function, get_colour_distribution, get_safe_name, get_colour_dir_name, get_magnitude_mask
from variables_config import get_config



In [2]:

def get_simulation_color(simulation):
    """Get standard color for each simulation"""
    color_map = {
        "IllustrisTNG": "blue",
        "SIMBA": "green",
        "Astrid": "red",
        "Swift-EAGLE": "orange"
    }
    return color_map.get(simulation, "gray")


In [3]:

def plot_cv_uvlf(input_dir, redshift_label, band, category, simulation):
    """Plot UVLF for CV simulations showing mean, scatter, and individual runs."""
    pattern = f"UVLF_CV_*_{get_safe_name(band)}_{redshift_label}_{category}.txt"
    files = glob.glob(os.path.join(input_dir, pattern))
    
    if not files:
        return None, None, None
    
    all_data = []
    for f in files:
        try:
            df = pd.read_csv(f, sep='\t')
            all_data.append(df)
        except Exception as e:
            continue
    
    if not all_data:
        return None, None, None
    
    magnitudes = all_data[0]['magnitude'].values
    phi_arrays = np.array([df['phi'].values for df in all_data])
    phi_sigma_arrays = np.array([df['phi_sigma'].values for df in all_data])
    
    mean_phi = np.mean(phi_arrays, axis=0)
    std_phi = np.std(phi_arrays, axis=0)
    lower_percentile = np.percentile(phi_arrays, 16, axis=0)
    upper_percentile = np.percentile(phi_arrays, 84, axis=0)
    
    sim_color = get_simulation_color(simulation)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    for phi in phi_arrays:
        ax.plot(magnitudes, phi, '-', color=sim_color, alpha=0.1)
    
    ax.plot(magnitudes, mean_phi, '-', color=sim_color, linewidth=2, 
            label=f'{simulation} Mean UVLF')
    ax.fill_between(magnitudes, lower_percentile, upper_percentile,
                   alpha=0.3, color=sim_color, label='16th-84th Percentile')
    
    ax.errorbar(magnitudes, mean_phi, yerr=std_phi/np.sqrt(len(all_data)),
               fmt='none', ecolor=sim_color, capsize=5)
    
    ax.set_xlabel('M$_{UV}$ [AB mag]', fontsize=12)
    ax.set_ylabel('log$_{10}$ φ [Mpc$^{-3}$ mag$^{-1}$]', fontsize=12)
    ax.set_title(f'{simulation} {band} UVLF\n{category}, {redshift_label}', fontsize=14)
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    return fig, magnitudes, mean_phi




In [4]:
def plot_cv_colours(input_dir, redshift_label, band1, band2, category, simulation):
    """Plot colour distributions for CV simulations."""
    filter_system = get_colour_dir_name(band1, band2)
    pattern = f"Colour_CV_*_{filter_system}_{redshift_label}_{category}.txt"
    files = glob.glob(os.path.join(input_dir, pattern))
    
    if not files:
        print(f"No files found in {input_dir} matching {pattern}")
        return None, None, None
    
    all_data = []
    for f in files:
        try:
            df = pd.read_csv(f, sep='\t')
            all_data.append(df)
        except Exception as e:
            print(f"Error reading {f}: {str(e)}")
            continue
    
    if not all_data:
        return None, None, None
    
    colours = all_data[0]['colour'].values
    dist_arrays = np.array([df['distribution'].values for df in all_data])
    mean_dist = np.mean(dist_arrays, axis=0)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    
    sim_color = get_simulation_color(simulation)
    for dist in dist_arrays:
        ax.plot(colours, dist, '-', color=sim_color, alpha=0.2)
    
    ax.plot(colours, mean_dist, '-', color=sim_color, linewidth=2, 
            label=simulation)
    
    ax.set_xlabel(f'{band1} - {band2} [mag]', fontsize=12)
    ax.set_ylabel('Normalized Count', fontsize=12)
    ax.set_xlim(-0.5, 3.5)
    ax.set_ylim(0, 0.7)
    ax.grid(True, alpha=0.3)
    ax.legend(frameon=True)
    
    return fig, colours, mean_dist

In [5]:

def plot_combined_colours(all_sims_data, redshifts, output_dir, colour_pairs):
    """Create combined colour plot showing all simulations."""
    fig, ax = plt.subplots(figsize=(10, 8))
    
    for sim_name, sim_data in all_sims_data.items():
        color = get_simulation_color(sim_name)
        if sim_data:
            # Get first colour pair's data (assuming we're just plotting FUV-NUV)
            colour_key = list(sim_data.keys())[0]
            # Get first redshift's data (or specific redshift if needed)
            z_data = list(sim_data[colour_key].values())[0]
            
            if 'intrinsic' in z_data:
                ax.plot(z_data['intrinsic']['colour'], 
                       z_data['intrinsic']['distribution'],
                       '-', color=color, label=sim_name)
    
    ax.set_xlabel('GALEX FUV - GALEX NUV [mag]', fontsize=12)
    ax.set_ylabel('Normalized Count', fontsize=12)
    ax.set_xlim(-0.5, 3.5)
    ax.set_ylim(0, 0.7)
    ax.grid(True, alpha=0.3)
    ax.legend(frameon=True)
    
    output_file = os.path.join(output_dir, 'combined_colours.pdf')
    plt.savefig(output_file, bbox_inches='tight', dpi=300)
    plt.close()

In [6]:
def plot_combined_uvlf(all_sims_data, redshifts, output_dir):
    """Create a multi-panel plot showing UVLFs for all simulations at different redshifts."""
    num_redshifts = len(redshifts)
    fig, axes = plt.subplots(1, num_redshifts, figsize=(16, 4))
    if num_redshifts == 1:
        axes = [axes]
    
    for ax, redshift_info in zip(axes, redshifts):
        z = redshift_info['redshift']
        
        for sim_name, sim_data in all_sims_data.items():
            color = get_simulation_color(sim_name)
            
            if z in sim_data:
                z_data = sim_data[z]
                
                # Plot intrinsic (dashed) without label
                if 'intrinsic' in z_data:
                    ax.plot(z_data['intrinsic']['magnitude'], 
                           z_data['intrinsic']['phi'],
                           '--', color=color, alpha=0.5)
                
                # Plot attenuated (solid) with single label for simulation
                if 'attenuated' in z_data:
                    ax.plot(z_data['attenuated']['magnitude'], 
                           z_data['attenuated']['phi'],
                           '-', color=color,
                           label=sim_name)  # Only one label per simulation
        
        ax.set_xlabel('M$_{AB}$', fontsize=10)
        ax.set_ylabel('$\phi$ [Mpc$^{-3}$ dex$^{-1}$]' if ax == axes[0] else '', fontsize=10)
        ax.set_ylim(-5.0, -2.0)
        ax.set_xlim(-26, -17)
        ax.grid(True, alpha=0.3)
        ax.text(0.05, 0.95, f'z = {z}', transform=ax.transAxes, 
                bbox=dict(facecolor='white', alpha=0.8, pad=0.1),
                fontsize=10)
        ax.tick_params(axis='both', which='major', labelsize=8)
    
    # Simpler legend with just simulation names
    legend = axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', 
                          fontsize=8, handlelength=1.5)
    
    plt.subplots_adjust(wspace=0.1)
    plt.savefig(os.path.join(output_dir, 'combined_uvlf.pdf'), 
                bbox_inches='tight', dpi=300)
    plt.close()

In [8]:
if __name__ == "__main__":
    simulations = ["IllustrisTNG", "SIMBA", "Astrid", "Swift-EAGLE"]
    
    # Dictionaries to store all data
    all_uvlf_data = {}
    all_colour_data = {}
    
    # Get base plots directory from config (do this once at the start)
    config = get_config(dataset="CV", simulation=simulations[0])
    base_plots_dir = os.path.join("/home/jovyan/camels/proj1/CV_set/CV_outputs/plots")  # Or however your base path should be constructed
    
    
    for simulation in simulations:
        print(f"\nProcessing simulation: {simulation}")
        config = get_config(dataset="CV", simulation=simulation)
        base_plot_dir = os.path.dirname(config["plots_dir"]["UVLFs"]["intrinsic"])
        print(f"Base plot directory: {base_plot_dir}")
        
        # Initialize simulation data
        sim_uvlf_data = {}
        sim_colour_data = {}  # Changed from sim_color_data
        
        # Process UVLFs
        for snap, redshift_info in config["redshift_values"].items():
            print(f"\nProcessing redshift: {redshift_info['label']}")
            z_data = {}
            
            for category in ['intrinsic', 'attenuated']:
                for band in config["filters"][category]:
                    print(f"Processing {category} {band}")
                    filter_system = get_safe_name(band, filter_system_only=True)
                    data_dir = os.path.join(config["lf_data_dir"][category][filter_system],
                                          get_safe_name(redshift_info['label']))
                    
                    # Create individual UVLF plot
                    fig, magnitudes, mean_phi = plot_cv_uvlf(
                        input_dir=data_dir,
                        redshift_label=redshift_info['label'],
                        band=band,
                        category=category,
                        simulation=simulation
                    )
                    
                    if fig is not None:
                        plot_output_dir = config["plots_dir"]["UVLFs"][category]
                        os.makedirs(plot_output_dir, exist_ok=True)
                        output_file = os.path.join(
                            plot_output_dir,
                            f"UVLF_{simulation}_{get_safe_name(band)}_{redshift_info['label']}_{category}.pdf"
                        )
                        print(f"Saving UVLF plot to: {output_file}")
                        fig.savefig(output_file, bbox_inches='tight', dpi=300)
                        plt.close(fig)
                    
                    # Store data for combined plot
                    if magnitudes is not None and mean_phi is not None:
                        if category not in z_data:
                            z_data[category] = {}
                        z_data[category]['magnitude'] = magnitudes
                        z_data[category]['phi'] = mean_phi
            
            sim_uvlf_data[redshift_info['redshift']] = z_data

        # Process colours for this redshift
        for category in ['intrinsic', 'attenuated']:
            for band1, band2 in config["colour_pairs"]:
                print(f"Processing colours {band1}-{band2} for {category}")
                filter_system = get_colour_dir_name(band1, band2)
                data_dir = os.path.join(config["colour_data_dir"][category],
                                      filter_system,
                                      get_safe_name(redshift_info['label']))
                print(f"Looking for colour data in: {data_dir}")
                
                fig, colours, mean_dist = plot_cv_colours(  # Changed function name and variable name
                    input_dir=data_dir,
                    redshift_label=redshift_info['label'],
                    band1=band1,
                    band2=band2,
                    category=category,
                    simulation=simulation
                )
                
                if fig is not None:
                    plot_output_dir = config["plots_dir"]["colours"][category]
                    os.makedirs(plot_output_dir, exist_ok=True)
                    output_file = os.path.join(
                        plot_output_dir,
                        f"Colour_{simulation}_{filter_system}_{redshift_info['label']}_{category}.pdf"
                    )
                    print(f"Saving colour plot to: {output_file}")
                    fig.savefig(output_file, bbox_inches='tight', dpi=300)
                    plt.close(fig)
                else:
                    print(f"No colour plot generated for {simulation} {band1}-{band2}")
                
                if colours is not None and mean_dist is not None:
                    if f'{band1}-{band2}' not in sim_colour_data:
                        sim_colour_data[f'{band1}-{band2}'] = {}
                    if redshift_info['redshift'] not in sim_colour_data[f'{band1}-{band2}']:
                        sim_colour_data[f'{band1}-{band2}'][redshift_info['redshift']] = {}
                    sim_colour_data[f'{band1}-{band2}'][redshift_info['redshift']][category] = {
                        'colour': colours,
                        'distribution': mean_dist
                    }
                    
        print(f"\nStoring data for {simulation}")
        all_uvlf_data[simulation] = sim_uvlf_data
        all_colour_data[simulation] = sim_colour_data  # Changed variable name
    
    # Create combined plots directory at the correct level
    combined_plot_dir = os.path.join(base_plots_dir, "combined")
    os.makedirs(combined_plot_dir, exist_ok=True)
    print(f"\nCreating combined plots in: {combined_plot_dir}")
    
    # Create combined plots
    print("Creating combined UVLF plot")
    plot_combined_uvlf(all_uvlf_data, 
                      sorted(config["redshift_values"].values(), key=lambda x: x['redshift']),
                      combined_plot_dir)
    
    print("Creating combined colours plot")
    plot_combined_colours(all_colour_data,
                        sorted(config["redshift_values"].values(), key=lambda x: x['redshift']),
                        combined_plot_dir,
                        config["colour_pairs"])


Processing simulation: IllustrisTNG
Base plot directory: /home/jovyan/camels/proj1/CV_set/CV_outputs/plots/IllustrisTNG/UVLFs

Processing redshift: z2.0
Processing intrinsic UV1500
Saving UVLF plot to: /home/jovyan/camels/proj1/CV_set/CV_outputs/plots/IllustrisTNG/UVLFs/intrinsic/UVLF_IllustrisTNG_UV1500_z2.0_intrinsic.pdf
Processing intrinsic GALEX FUV
Saving UVLF plot to: /home/jovyan/camels/proj1/CV_set/CV_outputs/plots/IllustrisTNG/UVLFs/intrinsic/UVLF_IllustrisTNG_GALEX_FUV_z2.0_intrinsic.pdf
Processing intrinsic GALEX NUV
Saving UVLF plot to: /home/jovyan/camels/proj1/CV_set/CV_outputs/plots/IllustrisTNG/UVLFs/intrinsic/UVLF_IllustrisTNG_GALEX_NUV_z2.0_intrinsic.pdf
Processing attenuated GALEX FUV
Saving UVLF plot to: /home/jovyan/camels/proj1/CV_set/CV_outputs/plots/IllustrisTNG/UVLFs/attenuated/UVLF_IllustrisTNG_GALEX_FUV_z2.0_attenuated.pdf
Processing attenuated GALEX NUV
Saving UVLF plot to: /home/jovyan/camels/proj1/CV_set/CV_outputs/plots/IllustrisTNG/UVLFs/attenuated/UVLF